In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.2 MB/s eta 0:00:00


In [3]:
#Import the necessary libraries

import openai
import typing
from typing import List, Tuple

import tiktoken
import ast

import pandas as pd
import time

Prompt Generator Code

In [4]:
def build_examples_prompt(concern_type, concerns_list, examples):
    prompt_builder = f"Give me at least 1000 examples. Augment data for NLP that indicate that there is a {concern_type} concern. Anything that the instructor should look at such as {', '.join(concerns_list)}\nhere are examples\n"
    example_template = "{}\n"


    for example in examples:
        prompt_builder += example_template.format(example)

    return prompt_builder

# Here are the examples I used
time_management_concerns = ["time management"]
examples_time = [
        "I struggle to keep track of all deadlines",
        "I have a tough time making all the deadlines on time",
        "Too much to keep track of",
        "I don't know how to cram all this knowledge in the limited time I have"
]

examples_other = [
    "I have cancer",
    "I am depressed",
    "I need accomodations for my ADHD",
    "I know people in this class who are cheating",
    "I have a question on how to access the textbook",
    "I need to discuss my 504 plan",
    "This class is giving me anxiety",
    "The tests give me anxiety and make me nervous, I have test anxiety",
    "I need help for my disability",
    "I need accomodations for my wheelechair disability",
    "Would it be okay to come in during office hours to ask for help with things that were covered in pre-requisite courses?",
    "Is there a list of all the vocabulary terms that are useful for this course?",
    "I would like to know if we are going to use any new notations on homework.",
    "Is it ok to work with other people to finish homework?",
    "I broke my hand and need help writing",
    "my mother has covid",
    "I have Covid",
    "I am feeling awful, nothing seems to work and I hate this course",
    "I want to contact the dean of students",
    "How do I set up a meeting with you?",
    "How do I get one on one tutoring?"

]

other_concerns = ["mental health issues", "OSI (office of student integrity)", "Dean of Students", "Disability", "Unstable", "Anxiety", "Accommodation", "ADH", "Disorder", "Depression", "Travel Plans", "504 plan", "any health concern", "any question"]
lm_concerns =["Difficulty Learning material", "Exam Stress", "Reviewing Pre Requisite material for calculus or some other previous class", "Any struggles with learning the material itself", "Finding material difficult"]
examples_lm = [
    "That I forget all my knowledge from calc BC",
    "My only concern about this course is that I will have an issue learning one of the fundamental items early in the course, and as a result of that, will struggle later on when building upon that foundational knowledge. ",
    "The material is really tough and its just not getting to my brain",
    "I'm struggling to learn, I just don't get it",
    "I don't rememeber calculus at all",
    "I don't know how to study for the exams, its so tough to review all that information :(",
    "I found topics related to series really tough last year and I feel like it might be tough to pick up on its content"


]
time_manage_prompt = build_examples_prompt("time management", time_management_concerns, examples_time)
other_prompt = build_examples_prompt("some other important concern", other_concerns, examples_other)
lm_prompt = build_examples_prompt("learning management concern", lm_concerns, examples_lm)
print("Time Management Prompt:")
print(time_manage_prompt)

print("\nOther Concern Prompt:")
print(other_prompt)

print("\nLearning management Concern Prompt:")
print(lm_prompt)

Time Management Prompt:
Give me at least 1000 examples. Augment data for NLP that indicate that there is a time management concern. Anything that the instructor should look at such as time management
here are examples
I struggle to keep track of all deadlines
I have a tough time making all the deadlines on time
Too much to keep track of
I don't know how to cram all this knowledge in the limited time I have


Other Concern Prompt:
Give me at least 1000 examples. Augment data for NLP that indicate that there is a some other important concern concern. Anything that the instructor should look at such as mental health issues, OSI (office of student integrity), Dean of Students, Disability, Unstable, Anxiety, Accommodation, ADH, Disorder, Depression, Travel Plans, 504 plan, any health concern, any question
here are examples
I have cancer
I am depressed
I need accomodations for my ADHD
I know people in this class who are cheating
I have a question on how to access the textbook
I need to discu

Set the OpenAI API Key

In [ ]:
#Sets the open A.I Key
def SetKeyWithFile(keyFile = '/Users/adi/Desktop/RandomCode/Confidential/ChatGPTApiKey.txt') -> str:
    #Opens the File with the API in it
    text = open(keyFile, 'r')
    myKey = text.read()
    text.close()
    #Sets API key to file location
    openai.api_key = myKey
    return myKey
count()

NameError: name 'count' is not defined

In [ ]:
import os
print(os.getcwd())

Actual Implmentation

## Hugging Face Pre-trained Model Implementation

In [5]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.nn.functional import softmax
import numpy as np
from sklearn.metrics import accuracy_score

In [7]:
def load_model_and_tokenizer(model_name='distilbert-base-uncased', labels=4):
    # Gets the tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    # Load the pre-trained model
    loaded_model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=labels)
    loaded_model.eval()  # Set the model to evaluation mode

    return tokenizer, loaded_model


def predict_with_loaded_model(input_text, tokenizer, loaded_model):
    # Tokenize and encode the text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass through the model
    with torch.no_grad():
        outputs = loaded_model(**inputs)

    return outputs


def predict_column(df, category_mapping, text_column, model_name='distilbert-base-uncased'):
    # Load model and tokenizer
    tokenizer, loaded_model = load_model_and_tokenizer(model_name)
    df[text_column] = df[text_column].astype(str)

    # Apply prediction function to each value in the specified column
    predictions = df[text_column].apply(lambda x: predict_with_loaded_model(x, tokenizer, loaded_model))

    # Extract predicted classes and probabilities
    df['Predicted_Class: ' + text_column] = predictions.apply(lambda x: torch.argmax(softmax(x.logits, dim=1), dim=1).item()).map(category_mapping)
    df['Probabilities: ' + text_column] = predictions.apply(lambda x: softmax(x.logits, dim=1).tolist())

    # Calculate confidence intervals (dummy example, actual method depends on model specifics)
    df['Confidence_Interval: ' + text_column] = predictions.apply(lambda x: np.percentile(softmax(x.logits, dim=1).numpy(), [2.5, 97.5], axis=1))

    return df

def calculate_accuracy(df, text_column, ground_truth_column):
    # Calculate accuracy if ground truth is available
    if ground_truth_column in df.columns:
        y_true = df[ground_truth_column]
        y_pred = df['Predicted_Class: ' + text_column].map({v: k for k, v in category_mapping.items()})
        accuracy = accuracy_score(y_true, y_pred)
        return accuracy
    else:
        return None

df = pd.read_csv("2023.02.StartOfSemester.Coded.csv")

# Define category mapping
category_mapping = {0: 'OT', 1: "NC", 2: "LM", 3: "TM"}

# Convert 'concerns category' and 'anything else category' to numerical values
df['ground_truth_concerns'] = df['concerns category'].map({v: k for k, v in category_mapping.items()})
df['ground_truth_anything_else'] = df['anything else category'].map({v: k for k, v in category_mapping.items()})

# Check for and handle NaN values in the ground truth columns
df['ground_truth_concerns'] = df['ground_truth_concerns'].fillna(-1).astype(int)
df['ground_truth_anything_else'] = df['ground_truth_anything_else'].fillna(-1).astype(int)

# Predict categories for the 'concerns' column
df = predict_column(df, category_mapping, "concerns")

# Predict categories for the 'anything else' column
df = predict_column(df, category_mapping, "anything else")

# Calculate accuracy if ground truth is available
accuracy_concerns = calculate_accuracy(df, "concerns", "ground_truth_concerns")
accuracy_anything_else = calculate_accuracy(df, "anything else", "ground_truth_anything_else")

# Print accuracy results
print(f"Accuracy for 'concerns': {accuracy_concerns}")
print(f"Accuracy for 'anything else': {accuracy_anything_else}")

# Save the results to an Excel file
path = 'result_testDataLabeled.xlsx'
df.to_excel(path, index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy for 'concerns': 0.03333333333333333
Accuracy for 'anything else': 0.74


## Train your own BERT Model

In [8]:
import pandas as pd
def read_list_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            # Read the file content as a string
            file_content_str = file.read()

            # Safely evaluate the string as a Python literal (list)
            file_contents = ast.literal_eval(file_content_str)

        return file_contents

    except FileNotFoundError:
        print(f"The file '{file_path}' does not exist.")
    except (SyntaxError, ValueError) as e:
        print(f"Error while evaluating the file content: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
def df_from_file(file_path):
    listy = read_list_from_file(file_path)

    category = file_path.replace(".txt", "")
    df = pd.DataFrame({'response': listy, 'category': [category] * len(listy)})
    return df
#Files is a list of files to add it from. Put it all in one dataframe
def multiple_df(files):
    dfs = []
    for file in files:
        dfs.append(df_from_file(file))

    result_df = pd.concat(dfs, ignore_index=True)
    return result_df


ret = multiple_df(["NC.txt", "timeManagement.txt", "OTHER.txt", "LM.txt"])
shuffled_df = ret.sample(frac=1).reset_index(drop=True)
shuffled_df

,response,category
0,I need to figure out an accommodation plan for...,LM
1,I need to talk to someone about seeking accomm...,LM
2,I'm feeling very overwhelmed by the course mat...,LM
3,I have been feeling down and need to talk to s...,LM
4,I'm overwhelmed with the number of tasks I hav...,timeManagement
...,...,...
28459,difficulty solved,NC
28460,I can never seem to find enough time to get ev...,timeManagement
28461,I am dealing with a family emergency and need ...,OTHER
28462,I need someone to talk to about my anxiety,OTHER


In [9]:
import pandas as pd
from transformers import TFAutoModel, BertTokenizer
import torch
#Loads my tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#Loads my data frame
df = shuffled_df

#Puts my data frame in a good format for ML
#Max length is 512
#encoded_data = tokenizer(df['response'].tolist(), padding=True, truncation=True, max_length=512, return_tensors='pt')
#labels = torch.tensor(df['category'].tolist())

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
shuffled_df
# Define a mapping dictionary
category_mapping = {'OTHER': 0, 'NC': 1, 'LM': 2, 'timeManagement':3}

# Apply the mapping using the map function
shuffled_df['category_int'] = df['category'].map(category_mapping)
shuffled_df

,response,category,category_int
0,I need to figure out an accommodation plan for...,LM,2
1,I need to talk to someone about seeking accomm...,LM,2
2,I'm feeling very overwhelmed by the course mat...,LM,2
3,I have been feeling down and need to talk to s...,LM,2
4,I'm overwhelmed with the number of tasks I hav...,timeManagement,3
...,...,...,...
28459,difficulty solved,NC,1
28460,I can never seem to find enough time to get ev...,timeManagement,3
28461,I am dealing with a family emergency and need ...,OTHER,0
28462,I need someone to talk to about my anxiety,OTHER,0


In [12]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# Load your DataFrame
df = shuffled_df

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text and encode labels
encoded_data = tokenizer(df['response'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')  # Reduced max_length for speed
labels = torch.tensor(df['category_int'].tolist())

# Train-Test Split
train_texts, test_texts, train_labels, test_labels = train_test_split(encoded_data['input_ids'],
                                                                     labels,
                                                                     test_size=0.2,
                                                                     random_state=42)

# Create DataLoader for training
train_dataset = TensorDataset(train_texts, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# Create DataLoader for testing
test_dataset = TensorDataset(test_texts, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['category_int'].unique()))

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 3
model.train()
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0
    for batch in tqdm(train_dataloader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        epoch_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"Average Epoch Loss: {avg_epoch_loss:.4f}")

# Evaluation loop (optional)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_dataloader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(input_ids=inputs)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


100%|██████████| 1424/1424 [02:40<00:00,  8.86it/s]


Average Epoch Loss: 0.3944
Epoch 2/3


100%|██████████| 1424/1424 [02:42<00:00,  8.78it/s]


Average Epoch Loss: 0.3119
Epoch 3/3


100%|██████████| 1424/1424 [02:41<00:00,  8.83it/s]

Average Epoch Loss: 0.2849


Test Accuracy: 0.8498


In [13]:
# Save the model to a file
torch.save(model.state_dict(), 'bert_model.pth')

# Load the model from the file
loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
loaded_model.load_state_dict(torch.load('bert_model.pth'))
loaded_model.eval()  # Set the model to evaluation model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e